# Electricity Usage Analysis

In [111]:
for name in dir():
    if not name.startswith("_"):
        del globals()[name]

In [112]:
import sys
!{sys.executable} -m pip install hvplot

In [139]:
# Libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import hvplot.pandas

from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [114]:
def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [118]:
df3 = pd.read_csv('df3.csv', header='infer')
df3.set_index('date', inplace=True)
df3['Hour'] = df3['Hour'].astype(str)

In [119]:
print(df3.shape)
df3.head()

(16459, 29)


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,v1,Weekday,Hour
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.20,48.900000,17.033333,45.53,6.600000,733.5,92.000000,7.000000,63.000000,5.3,13.275433,Mon,17
2016-01-11 17:10:00,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,21.79,48.863333,18.890000,45.56,6.483333,733.6,92.000000,6.666667,40.000000,5.2,18.606195,Mon,17
2016-01-11 17:20:00,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.20,48.730000,17.000000,45.50,6.366667,755.0,92.000000,6.333333,55.333333,5.1,28.642668,Mon,17
2016-01-11 17:30:00,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.10,48.590000,17.000000,45.40,6.250000,733.8,84.333333,6.000000,51.500000,5.0,45.410390,Mon,17
2016-01-11 17:40:00,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,34.927778,18.10,48.590000,17.000000,45.40,6.133333,733.9,84.333333,5.666667,47.666667,4.9,10.084097,Mon,17


In [120]:
df3.describe()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,v1
count,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.000000,16459.00000,16459.000000,16459.000000
mean,96.233064,3.955283,21.411526,40.169764,20.036250,40.440233,21.935220,39.375334,20.468408,39.095497,19.286999,51.174386,7.166662,59.619036,19.787542,35.457365,21.692460,43.134914,19.029985,41.566813,6.848526,754.794260,80.719657,4.120142,38.61759,3.335968,24.934725
std,102.028297,8.060528,1.389577,3.513932,1.875805,3.643092,1.668433,3.038812,1.807956,3.971781,1.548120,8.694324,5.296161,28.978911,1.760854,4.457492,1.717435,4.855274,1.674996,3.856741,4.901699,6.937394,14.110826,2.358126,10.59493,3.680742,14.517159
min,10.000000,0.000000,16.790000,27.023333,16.100000,20.596667,17.200000,30.663333,15.100000,27.660000,15.330000,29.815000,-6.030000,1.000000,15.390000,23.200000,16.362222,29.600000,14.890000,29.166667,-5.000000,729.300000,24.000000,0.000000,1.00000,-6.600000,0.005322
25%,50.000000,0.000000,20.790000,37.900000,18.890000,38.400000,20.823333,37.290000,19.390000,36.290000,18.200000,46.090000,3.933333,40.725000,18.700000,32.500000,20.790000,39.568036,18.000000,38.976500,3.663889,751.100000,73.000000,2.333333,32.50000,1.166667,12.425313
50%,60.000000,0.000000,21.390000,39.626667,19.760000,40.590000,21.890000,38.730000,20.390000,38.500000,19.260000,49.400000,6.656667,62.261667,19.600000,34.927778,21.790000,42.600000,18.890000,41.000000,6.400000,755.000000,84.333333,3.833333,40.00000,3.200000,24.774977
75%,100.000000,0.000000,22.100000,42.433333,20.790000,42.826667,22.790000,41.290000,21.390000,41.465000,20.200000,53.334000,9.726667,84.945000,20.926667,37.905409,22.730000,46.060000,20.251429,43.760000,9.600000,759.266667,91.166667,5.333333,40.00000,5.486111,37.583769
max,1080.000000,70.000000,25.700000,63.360000,29.856667,56.026667,27.600000,50.163333,26.200000,51.090000,25.745000,96.321667,28.236000,99.900000,25.390000,51.400000,27.230000,58.780000,24.200000,53.326667,26.100000,772.300000,100.000000,14.000000,66.00000,15.500000,49.996530


In [203]:
# some testing ...
if 0:
    x = df3['Appliances'] - df3['Appliances'].min() + 1
    print(x[0:10].tolist())

    xbox, opt_lambda = boxcox(x)
    print(xbox[0:10].tolist())
    print(opt_lambda)

    xinvbox = inv_boxcox(xbox, opt_lambda)
    print(xinvbox[0:10].tolist())

[51.0, 51.0, 41.0, 41.0, 51.0, 41.0, 51.0, 51.0, 51.0, 61.0]
[2.646922942509263, 2.646922942509263, 2.5515385251450624, 2.5515385251450624, 2.646922942509263, 2.5515385251450624, 2.646922942509263, 2.646922942509263, 2.646922942509263, 2.7218760785681684]
-0.21655783519823987
[51.00000000000002, 51.00000000000002, 41.00000000000001, 41.00000000000001, 51.00000000000002, 41.00000000000001, 51.00000000000002, 51.00000000000002, 51.00000000000002, 60.99999999999995]


In [121]:
# df3
def get_filtered_df(excluded_cols):
    features = [col for col in df3.columns if col not in excluded_cols]
    X = df3[features]
    y = df3['Appliances']
    return (X, y)

In [122]:
def log_transform(v, subtract_min=1):
    vt = v
    if (subtract_min):
        vt = v - v.min() + 1
    return np.log(vt)

In [123]:
def boxcox_transform(v, subtract_min=1):
    v1 = v
    vmin = v.min()
    if (subtract_min):
        v1 = v - vmin + 1
    vt, opt_lambda = boxcox(v1)
    return (vt, vmin, opt_lambda)

In [124]:
# Box-Cox transformation of selected features in a dataaframe
def boxcox_transform_features(X, features):
    Xt = X.copy()
    opt_lambda_dict = {}
    for col in features:
        vt, vmin, opt_lambda = boxcox_transform(X[col])
        Xt[col] = vt
        opt_lambda_dict[col] = opt_lambda

    return (Xt, opt_lambda_dict)

In [152]:
def one_hot_encoding(X, cat_col_list):
    df_cat = X[cat_col_list]

    encoding_cat = OneHotEncoder()
    encoding_cat.fit(df_cat)
    df_cat_enc = pd.DataFrame(
        encoding_cat.transform(df_cat).toarray(), columns=encoding_cat.get_feature_names()
    )

    df_cat_enc.set_index(df_cat.index, inplace=True)
    Xt = pd.merge(X, df_cat_enc, how='left', left_index=True, right_index=True)
    Xt = Xt.drop(cat_col_list, axis=1)
    
    return (Xt)

In [125]:
#scoring = make_scorer(r2_score)
scoring = make_scorer(root_mean_squared_error, greater_is_better=False)

In [126]:
#-------------------------------------------------------------------------------
# Execute the cross-validation for all parameter combinations and plots the
# evaluation metrics. At each iteration of the cross validation process, the 
# model is evaluated using all the below metrics but the optimal model is 
# determined using the AUC score (as this minimizes both false positives &
# false negatives).
#-------------------------------------------------------------------------------
def build_optimal_model(model_type, model, model_parameter_grid, X, y, scoring):
    grid_object = GridSearchCV(model, param_grid=model_parameter_grid, cv=5, scoring=scoring, refit=True, return_train_score=True, n_jobs=-1)
    print('Performing grid search to determine the optimal parameters ...')
    grid_object.fit(X, y)
    grid_search_output = pd.DataFrame(grid_object.cv_results_)

    optimal_model = grid_object.best_estimator_
    optimal_model = optimal_model.fit(X, y)

    return grid_object, grid_search_output, optimal_model

In [127]:
# Decision Tree setup
model_type = 'Decision Tree'
model = DecisionTreeRegressor(random_state=1)
MAX_HEIGHT = 20
model_parameter_grid = {
         'max_depth': range(1,(MAX_HEIGHT+1)),
         'min_samples_leaf': [25, 50, 75, 100, 125, 150, 200]
        }

In [128]:
%%time

# 1. log transform of Appliances variable

excluded_cols = ['Appliances', 'Weekday', 'Hour']
X, y = get_filtered_df(excluded_cols)

Xt = X
yt = log_transform(y, subtract_min=0)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 4.09 s, sys: 616 ms, total: 4.71 s
Wall time: 15.2 s


In [129]:
#print(grid_object)
# grid_search_output.head()
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': X.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

0.6293092912974575
{'max_depth': 3, 'min_samples_leaf': 150}


,feature,importance
0,lights,0.526222
21,RH_out,0.270466
16,RH_8,0.107139
17,T9,0.049911
15,T8,0.046262
1,T1,0.000000
24,Tdewpoint,0.000000
23,Visibility,0.000000
22,Windspeed,0.000000
20,Press_mm_hg,0.000000


In [130]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.6040362362713255
99.59114559685756
0.1593092810695228
0.04714554421479755


In [131]:
%%time

# 2. Box-Cox transformation of features

excluded_cols = ['Appliances', 'Weekday', 'Hour']
X, y = get_filtered_df(excluded_cols)

Xt, opt_lambda_dict = boxcox_transform_features(X, X.columns.tolist())
yt, _, opt_lambda = boxcox_transform(y, subtract_min=0)
opt_lambda_dict['Appliances'] = opt_lambda # Appliances > 0

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...
CPU times: user 2.98 s, sys: 97.9 ms, total: 3.08 s
Wall time: 10.3 s


In [132]:
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': X.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

0.06127056195251598
{'max_depth': 5, 'min_samples_leaf': 150}


,feature,importance
0,lights,0.347718
17,T9,0.221971
21,RH_out,0.128043
16,RH_8,0.064109
15,T8,0.047725
11,T6,0.045741
2,RH_1,0.032181
18,RH_9,0.028905
3,T2,0.028105
13,T7,0.024467


In [133]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(inv_boxcox(predicted, opt_lambda_dict['Appliances']), y))

print(r2_score(yt, predicted))
print(r2_score(y, inv_boxcox(predicted, opt_lambda_dict['Appliances'])))

0.05543442469583618
100.20835154069583
0.24109657974490806
0.03529851106666693


In [155]:
%%time

# 3. One-hot encoding of Weekday & Hour features

excluded_cols = ['Appliances']
X, y = get_filtered_df(excluded_cols)

Xt = one_hot_encoding(X, ['Weekday', 'Hour'])
yt = log_transform(y, subtract_min=0)

#print(Xt.shape)
#Xt.head()

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 8.32 s, sys: 1.49 s, total: 9.81 s
Wall time: 25.2 s


In [157]:
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

0.6331328687969251
{'max_depth': 3, 'min_samples_leaf': 150}


,feature,importance
0,lights,0.493709
21,RH_out,0.206995
16,RH_8,0.104388
43,x1_18,0.092325
17,T9,0.057508
15,T8,0.045074
51,x1_4,0.000000
50,x1_3,0.000000
33,x1_0,0.000000
34,x1_1,0.000000


In [158]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(np.exp(predicted), np.exp(yt)))

print(r2_score(yt, predicted))
print(r2_score(np.exp(yt), np.exp(predicted)))

0.602526334180205
99.1398622788039
0.1635069569558042
0.05576143188041982


In [161]:
%%time

# 2. Box-Cox transformation of features

excluded_cols = ['Appliances']
X, y = get_filtered_df(excluded_cols)

cat_col_list = ['Weekday', 'Hour']
features_to_transform = [col for col in X.columns if col not in cat_col_list]
Xt1, opt_lambda_dict = boxcox_transform_features(X, features_to_transform)
yt, _, opt_lambda = boxcox_transform(y, subtract_min=0)
opt_lambda_dict['Appliances'] = opt_lambda # Appliances > 0

Xt = one_hot_encoding(Xt1, cat_col_list)

grid_object, grid_search_output, optimal_model = build_optimal_model(model_type, model, model_parameter_grid, Xt, yt, scoring)

Performing grid search to determine the optimal parameters ...


/Users/parthankasarapu/anaconda2/envs/py36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


CPU times: user 7.52 s, sys: 1.13 s, total: 8.65 s
Wall time: 22.7 s


In [162]:
print(-grid_object.best_score_)
print(grid_object.best_params_)
pd.DataFrame.from_dict({'feature': Xt.columns.tolist(), 'importance': optimal_model.feature_importances_}).sort_values('importance', ascending=False)

0.060120265880427286
{'max_depth': 5, 'min_samples_leaf': 150}


,feature,importance
0,lights,0.341449
17,T9,0.209065
21,RH_out,0.125734
16,RH_8,0.053347
15,T8,0.046865
11,T6,0.044916
43,x1_18,0.042521
22,Windspeed,0.029502
44,x1_19,0.029429
46,x1_20,0.026036


In [163]:
# model evaluation
predicted = optimal_model.predict(Xt)

print(root_mean_squared_error(predicted, yt))
print(root_mean_squared_error(inv_boxcox(predicted, opt_lambda_dict['Appliances']), y))

print(r2_score(yt, predicted))
print(r2_score(y, inv_boxcox(predicted, opt_lambda_dict['Appliances'])))

0.05527253166599446
99.90855126899221
0.24552277412720713
0.041062204963070004
